In [1]:
# Cell 1

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv("SeoulBikeData.csv", encoding='latin1')

df.head()


,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [2]:
# Cell 2

# Remove Date column
df.drop(columns=['Date'], inplace=True)

# Label encode non-numeric categorical columns if any
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = pd.factorize(df[col])[0]

# Features and target
X = df.drop("Rented Bike Count", axis=1).values
y = df["Rented Bike Count"].values.reshape(-1, 1)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Cell 3

class LinearRegressionMomentum:
    def __init__(self, lr=0.01, epochs=2000, beta=0.9):
        self.lr = lr
        self.epochs = epochs
        self.beta = beta

    def fit(self, X, y):
        m, n = X.shape
        
        # Initialize weights + momentum terms
        self.W = np.zeros((n, 1))
        self.b = 0
        
        vw = np.zeros((n, 1))
        vb = 0

        for _ in range(self.epochs):
            # Predictions
            y_pred = X @ self.W + self.b

            # Gradients
            dw = (1/m) * (X.T @ (y_pred - y))
            db = (1/m) * np.sum(y_pred - y)

            # Momentum updates
            vw = self.beta * vw + (1 - self.beta) * dw
            vb = self.beta * vb + (1 - self.beta) * db

            # Weight updates
            self.W -= self.lr * vw
            self.b -= self.lr * vb

    def predict(self, X):
        return X @ self.W + self.b


In [4]:
# Cell 4

model = LinearRegressionMomentum(lr=0.01, epochs=3000, beta=0.9)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [5]:
# Cell 5

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (ss_res/ss_tot)

print("RMSE:", rmse(y_test, y_pred))
print("MAE:", mae(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


RMSE: 443.9743002485317
MAE: 332.1377432372127
R²: 0.5269051741225371
